In [6]:
import pandas as pd
import numpy as np
import torch
import os
import scripts
from functools import lru_cache
import networkx as nx
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)  # Set a higher number if needed
pd.set_option('display.max_columns', None)


In [9]:
annotated1=pd.read_csv("/work/haarscheid/cancer_baseline2/cancer_baseline/output/annotated_matrices/max/annotated_attention_all_attention_instance_127.csv")

print(annotated1.shape)
print(annotated1.head())


                       

(4096, 4097)
  Unnamed: 0     AAGAB      AAMP    AARSD1     ABCA1     ABCA2     ABCA3  \
0      AAGAB  0.025756  0.000665  0.000103  0.000288  0.000190  0.000367   
1       AAMP  0.006652  0.010850  0.000341  0.000303  0.000824  0.000473   
2     AARSD1  0.006489  0.000351  0.045737  0.000639  0.000172  0.000407   
3      ABCA1  0.006649  0.000277  0.001276  0.046860  0.000201  0.002909   
4      ABCA2  0.006266  0.001194  0.000484  0.000477  0.037854  0.000414   

      ABCA7    ABCA12     ABCB1     ABCB8     ABCB9    ABCB10     ABCC1  \
0  0.000105  0.000359  0.001387  0.000277  0.000118  0.000646  0.000653   
1  0.000110  0.000149  0.000934  0.000506  0.000246  0.000675  0.000540   
2  0.000355  0.000195  0.000132  0.000351  0.000401  0.000664  0.000356   
3  0.000221  0.000132  0.001129  0.000431  0.000434  0.000848  0.000331   
4  0.000450  0.000690  0.001775  0.001304  0.000273  0.000658  0.000521   

      ABCC3     ABCC4     ABCC5     ABCC9    ABCC10     ABCD1     ABCE1  \
0  0

In [3]:
if 'flag' not in globals():
    flag = False

In [ ]:
if not flag:
    # Convert filtered RNA data to tensor format
    tensor_exp = torch.Tensor(filtered_rna.to_numpy())
    cell_dict = {cell: tensor_exp[i] for i, cell in enumerate(filtered_rna.index.to_numpy())}

    # Load and filter drug response data
    data = pd.read_csv("data/GDSC1.csv", index_col=0)
    data = data.query("SANGER_MODEL_ID in @cell_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()

    # Set the flag to prevent reloading
    flag = True
else:
    print("Data already loaded, skipping.")

In [5]:
edge_weights_attention="/work/haarscheid/cancer_baseline2/cancer_baseline/output/annotated_matrices/mean/annotated_attention_all_attention_instance_20.csv"
attention_df = pd.read_csv(edge_weights_attention,index_col=0)

print(attention_df.shape)


attention_df = attention_df[~attention_df.index.str.startswith("ENSG")]
attention_df = attention_df.loc[:, ~attention_df.columns.str.startswith("ENSG")]
print(attention_df.shape)
filtered_gene_list = filtered_rna.columns.tolist()




(763, 763)
(739, 739)


In [6]:
# Assuming `filtered_rna` and `attention_df` are already defined
full_gene_list = filtered_rna.columns.tolist()

# Create an empty DataFrame filled with zeros, with the same structure as needed
expanded_attention_matrix = pd.DataFrame(
    0.0,  # Fill with zeros initially
    index=full_gene_list,
    columns=full_gene_list
)

# Copy over data from `attention_df` to the new expanded DataFrame
# Ensure that `attention_df` has both rows and columns in `full_gene_list`
common_genes = attention_df.index.intersection(full_gene_list)

# Vectorized assignment for improved performance
expanded_attention_matrix.loc[common_genes, full_gene_list] = attention_df.loc[common_genes, :]
expanded_attention_matrix.loc[full_gene_list, common_genes] = attention_df.loc[:, common_genes]

# Convert the expanded attention matrix to a PyTorch tensor
edge_weights = torch.tensor(expanded_attention_matrix.values, dtype=torch.float)

# Print the new attention matrix to check the structure
print("Expanded attention matrix with zero-filled rows and columns:")
#display(expanded_attention_matrix)
print(expanded_attention_matrix.shape)

Expanded attention matrix with zero-filled rows and columns:
(777, 777)


In [7]:

for idx, key in enumerate(cell_dict.keys()):
    if idx == 0:  # Only process the first key
        instance = cell_dict[key]
        break
node_weights = torch.tensor(instance, dtype=torch.float)
len(node_weights)



/tmp/ipykernel_2040686/907805343.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  node_weights = torch.tensor(instance, dtype=torch.float)


777

In [8]:
# Convert the `attention_df` to a tensor if not already done
adjacency_matrix = torch.tensor(expanded_attention_matrix.to_numpy(), dtype=torch.float)

# Extract the indices of non-zero entries (i.e., existing directed edges)
row_indices, col_indices = torch.nonzero(adjacency_matrix, as_tuple=True)
edge_indices = torch.stack([row_indices, col_indices], dim=0)  # Shape: [2, num_edges]

# Extract the actual weights of these edges
edge_attr = adjacency_matrix[row_indices, col_indices]  # Shape: [num_edges]

# Ensure `node_weights` is defined; create it as all ones if node features are not specified
node_weights = torch.ones((adjacency_matrix.size(0), 1), dtype=torch.float)  # Shape: [num_nodes, num_node_features]

# Create the PyTorch Geometric Data object
graph_data = Data(x=node_weights, edge_index=edge_indices, edge_attr=edge_attr)

# Print the graph data to check the setup
print("Constructed Graph Data Object:")
print(graph_data)

Constructed Graph Data Object:
Data(x=[777, 1], edge_index=[2, 602285], edge_attr=[602285])


In [17]:
print(filtered_rna.columns)

Index(['ABCB1', 'ABI1', 'ABL1', 'ABL2', 'ACKR3', 'ACSL3', 'ACVR1', 'ACVR2A',
       'AFDN', 'AFF1',
       ...
       'ZNF429', 'ZNF521', 'ZNF626', 'ZNF680', 'ZNF721', 'ZNF780A', 'ZNF814',
       'ZNRF3', 'ZRSR2', 'ZXDB'],
      dtype='object', name='Unnamed: 1', length=777)
